In [2]:
from roboflow import Roboflow
rf = Roboflow(api_key="Agk88hbUOjF1YUs8mxWw")
project = rf.workspace("capstone-x2wm7").project("hama-tanaman-berdaun-cy6p5")
version = project.version(5)
dataset = version.download("tensorflow")
                

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Hama-tanaman-berdaun-5 in tensorflow:: 100%|██████████| 2038/2038 [00:04<00:00, 498.12it/s]


In [59]:
# Example paths - replace in your code as needed
train_path = "Hama-tanaman-berdaun-5/train"
test_path = "Hama-tanaman-berdaun-5/test"
valid_path = "Hama-tanaman-berdaun-5/valid"

In [60]:
import os
import tensorflow as tf

In [61]:
def load_image_and_label(img_path, label_path):
    img = tf.io.read_file(img_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, [224, 224]) 
    img = tf.image.convert_image_dtype(img, tf.float32) 

    labels = parse_annotations(label_path)
    return img, labels

In [62]:
def parse_annotations(label_path):
    return tf.constant([0, 1, 0, 0, 0, 0, 0], dtype=tf.float32)

In [63]:
train_dataset = tf.data.Dataset.list_files(os.path.join(train_path, "*.jpg"))
train_dataset = train_dataset.map(
    lambda x: load_image_and_label(x, tf.strings.regex_replace(x, ".jpg$", ".xml")),
    num_parallel_calls=tf.data.AUTOTUNE
)

In [64]:
train_dataset = train_dataset.shuffle(buffer_size=1000).batch(32).prefetch(tf.data.AUTOTUNE)

In [65]:
# Create the validation dataset
valid_dataset = tf.data.Dataset.list_files(os.path.join(valid_path, "*.jpg"))
valid_dataset = valid_dataset.map(
    lambda x: load_image_and_label(x, tf.strings.regex_replace(x, ".jpg$", ".xml")),
    num_parallel_calls=tf.data.AUTOTUNE
)

# Shuffle, batch, and prefetch the validation dataset
valid_dataset = valid_dataset.batch(32).prefetch(tf.data.AUTOTUNE)

In [66]:
import tensorflow as tf
from tensorflow.keras import layers, models

def create_model():
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dense(7, activation='softmax')  # Replace 7 with the number of classes in your dataset
    ])
    return model

In [68]:
model = create_model()
model.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

model.fit(train_dataset, epochs=10, validation_data=valid_dataset)   # Adjust as needed

Epoch 1/10
60/60 [==============================] - 34s 462ms/step - loss: 2.0417 - accuracy: 0.9831 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 2/10
60/60 [==============================] - 28s 457ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 3/10
60/60 [==============================] - 28s 450ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 4/10
60/60 [==============================] - 29s 458ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 5/10
60/60 [==============================] - 28s 460ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 6/10
60/60 [==============================] - 28s 460ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 7/10
60/60 [==============================] - 27s 448ms/step - loss: 0.0000e+00 - accuracy: 

In [71]:
import matplotlib.pyplot as plt

test_img_path = "hama-tanaman-berdaun-cy6p5/test/sample_image.jpg"
img = tf.io.read_file(test_img_path)
img = tf.image.decode_jpeg(img, channels=3)
img = tf.image.resize(img, [224, 224]) 
img = tf.expand_dims(img, axis=0)

predictions = model.predict(img)
print(predictions)

plt.imshow(img[0].numpy())
plt.show()


NotFoundError: {{function_node __wrapped__ReadFile_device_/job:localhost/replica:0/task:0/device:CPU:0}} NewRandomAccessFile failed to Create/Open: hama-tanaman-berdaun-cy6p5/test/sample_image.jpg : The system cannot find the path specified.
; No such process [Op:ReadFile]

In [ ]:
model.save("hama_detection_model.h5")